In [23]:
import pandas as pd
import json
import re
import plotly.express as px
import numpy as np
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)

### RQ2.6
varying the values (context) of the parameters in both the then and then clauses

In [24]:
current_result = "6_shared_scenarios_results"

In [25]:
file_path = f'res/outcome/similarities/{current_result}.csv'
df = pd.read_csv(file_path, sep=';')

In [26]:
df.drop(columns=['Desired When DNF',
       'Shared When DNF', 'Penality When', 'Global Similarity When',
       'Final Similarity When'], inplace=True)
df

,Scenario,Desired Given DNF,Shared Given DNF,Penality Given,Global Similarity Given,Final Similarity Given,Desired Then DNF,Shared Then DNF,Penality Then,Global Similarity Then,Final Similarity Then,Scenario Similarity,Action
0,Candidate_1,battery >= 10 AND satellite_count >= 7 AND win...,battery >= 10 AND satellite_count >= 7 AND win...,0.0,1.000000,1.000000,height >= 30 AND source_distance == 0,height >= 30 AND source_distance == 0,0.0,1.000000,1.000000,1.000000,behavior_1
1,Candidate_1081,battery >= 10 AND satellite_count >= 7 AND win...,battery >= 0 AND satellite_count >= 7 AND wind...,0.0,0.966667,0.966667,height >= 30 AND source_distance == 0,height >= 30 AND source_distance == 0,0.0,1.000000,1.000000,0.988889,behavior_1081
2,Candidate_361,battery >= 10 AND satellite_count >= 7 AND win...,battery >= 10 AND satellite_count >= 8 AND win...,0.0,0.958333,0.958333,height >= 30 AND source_distance == 0,height >= 30 AND source_distance == 0,0.0,1.000000,1.000000,0.986111,behavior_361
3,Candidate_1441,battery >= 10 AND satellite_count >= 7 AND win...,battery >= 0 AND satellite_count >= 8 AND wind...,0.0,0.925000,0.925000,height >= 30 AND source_distance == 0,height >= 30 AND source_distance == 0,0.0,1.000000,1.000000,0.975000,behavior_1441
4,Candidate_11,battery >= 10 AND satellite_count >= 7 AND win...,battery >= 10 AND satellite_count >= 7 AND win...,0.0,1.000000,1.000000,height >= 30 AND source_distance == 0,height >= 50 AND source_distance == 0,0.0,0.857145,0.857145,0.952382,behavior_11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,Candidate_3887,battery >= 10 AND satellite_count >= 7 AND win...,battery <= 10 AND satellite_count <= 7 AND win...,0.0,0.000000,0.000000,height >= 30 AND source_distance == 0,height <= 30 AND source_distance >= 0,0.0,0.005000,0.005000,0.335000,behavior_3887
5396,Candidate_3948,battery >= 10 AND satellite_count >= 7 AND win...,battery <= 10 AND satellite_count <= 7 AND win...,0.0,0.000000,0.000000,height >= 30 AND source_distance == 0,height <= 30 AND source_distance >= 50,0.0,0.000000,0.000000,0.333333,behavior_3948
5397,Candidate_3888,battery >= 10 AND satellite_count >= 7 AND win...,battery <= 10 AND satellite_count <= 7 AND win...,0.0,0.000000,0.000000,height >= 30 AND source_distance == 0,height <= 30 AND source_distance >= 50,0.0,0.000000,0.000000,0.333333,behavior_3888
5398,Candidate_4128,battery >= 10 AND satellite_count >= 7 AND win...,battery <= 10 AND satellite_count <= 0 AND win...,0.0,0.000000,0.000000,height >= 30 AND source_distance == 0,height <= 30 AND source_distance >= 50,0.0,0.000000,0.000000,0.333333,behavior_4128


In [27]:
def extract_clauses(expression):
    # Primeiro, dividir a expressão usando 'AND' como delimitador
    clauses = expression.split(' AND ')
    
    # Regex para capturar o padrão: variável, operador, e valor, incluindo == e !=
    pattern = r'(\w+)\s*(>=|<=|>|<|=|==|!=)\s*([\d.]+)'
    
    # Extrair e converter as cláusulas em um dicionário
    result = {}
    for clause in clauses:
        match = re.match(pattern, clause.strip())
        if match:
            var, op, val = match.groups()
            val = float(val)
            result[var] = (var, op, val)
    
    return result


# Função ajustada para criar pares de cláusulas com as colunas desejadas
def create_pairs(row, col1, col2):
    clauses_1 = extract_clauses(row[col1])
    clauses_2 = extract_clauses(row[col2])
    
    # Obter o conjunto de todas as variáveis únicas das duas colunas
    all_vars = set(clauses_1.keys()).union(set(clauses_2.keys()))
    
    # Criar o vetor de duplas
    pairs = [(clauses_1.get(var, ('', '', '')), clauses_2.get(var, ('', '', ''))) for var in all_vars]
    
    return pairs


# Aplicar a função para criar o vetor de duplas para cada linha do DataFrame
df['Pairs_Given'] = df.apply(lambda row: create_pairs(row, 'Desired Given DNF', 'Shared Given DNF'), axis=1)
df['Pairs_Then'] = df.apply(lambda row: create_pairs(row, 'Desired Then DNF', 'Shared Then DNF'), axis=1)

# Exibir o DataFrame com as colunas de pares criadas
df[['Pairs_Given', 'Pairs_Then']]


,Pairs_Given,Pairs_Then
0,"[((satellite_count, >=, 7.0), (satellite_count...","[((height, >=, 30.0), (height, >=, 30.0)), ((s..."
1,"[((satellite_count, >=, 7.0), (satellite_count...","[((height, >=, 30.0), (height, >=, 30.0)), ((s..."
2,"[((satellite_count, >=, 7.0), (satellite_count...","[((height, >=, 30.0), (height, >=, 30.0)), ((s..."
3,"[((satellite_count, >=, 7.0), (satellite_count...","[((height, >=, 30.0), (height, >=, 30.0)), ((s..."
4,"[((satellite_count, >=, 7.0), (satellite_count...","[((height, >=, 30.0), (height, >=, 50.0)), ((s..."
...,...,...
5395,"[((satellite_count, >=, 7.0), (satellite_count...","[((height, >=, 30.0), (height, <=, 30.0)), ((s..."
5396,"[((satellite_count, >=, 7.0), (satellite_count...","[((height, >=, 30.0), (height, <=, 30.0)), ((s..."
5397,"[((satellite_count, >=, 7.0), (satellite_count...","[((height, >=, 30.0), (height, <=, 30.0)), ((s..."
5398,"[((satellite_count, >=, 7.0), (satellite_count...","[((height, >=, 30.0), (height, <=, 30.0)), ((s..."


In [28]:
# Carregando os limites das variáveis a partir do JSON
with open('res/variables_config.json', 'r') as file:
    variable_configs = json.load(file)

# Criar um dicionário com os limites de cada variável
limits = {var['name']: {'min': var['min_value'], 'max': var['max_value']} for var in variable_configs}

# Função para calcular o intervalo baseado no operador
def calculate_interval(var, op, val):
    min_limit, max_limit = limits[var]['min'], limits[var]['max']
    
    if op == '>=':
        return (val, max_limit)
    elif op == '<=':
        return (min_limit, val)
    elif op == '>':
        return (val, max_limit)
    elif op == '<':
        return (min_limit, val)
    elif op == '==':
        return (val, val)
    elif op == '!=':
        return (min_limit, max_limit)  # assume o escopo completo
    else:
        raise ValueError(f"Operador '{op}' não reconhecido.")

# Função para calcular o erro característico entre duas tuplas
def calculate_scope_error(pair):
    tuple1, tuple2 = pair
    
    if tuple1[0] != tuple2[0]:  # Verifica se as variáveis são diferentes
        return f"Erro: Variáveis diferentes {tuple1[0]} e {tuple2[0]}"
    
    var = tuple1[0]
    interval1 = calculate_interval(*tuple1)
    interval2 = calculate_interval(*tuple2)
    
    # Calculando o erro característico
    scope_error = (abs(interval1[0] - interval2[0]), abs(interval1[1] - interval2[1]))
    return scope_error

# Exemplo de DataFrame com pares
# data = {
#     'Pairs': [[(('battery', '>=', 10.0), ('battery', '<=', 50.0))],
#               [(('satellite_count', '>=', 5), ('satellite_count', '<=', 10))],
#               [(('wind_speed', '>', 20.0), ('wind_speed', '<', 30.0))]]
# }

# df = pd.DataFrame(data)

# Aplicar a função para calcular o erro característico para cada par de tuplas em 'Pairs'
df['Scope Error by var given'] = df['Pairs_Given'].apply(lambda pairs: [calculate_scope_error(pair) for pair in pairs])
df['Scope Error by var then'] = df['Pairs_Then'].apply(lambda pairs: [calculate_scope_error(pair) for pair in pairs])

# Exibir o DataFrame com os erros característicos
df[['Pairs_Given', 'Scope Error by var given']].head()

,Pairs_Given,Scope Error by var given
0,"[((satellite_count, >=, 7.0), (satellite_count...","[(0.0, 0), (0.0, 0.0), (0.0, 0.0)]"
1,"[((satellite_count, >=, 7.0), (satellite_count...","[(0.0, 0), (10.0, 0.0), (0.0, 0.0)]"
2,"[((satellite_count, >=, 7.0), (satellite_count...","[(1.0, 0), (0.0, 0.0), (0.0, 0.0)]"
3,"[((satellite_count, >=, 7.0), (satellite_count...","[(1.0, 0), (10.0, 0.0), (0.0, 0.0)]"
4,"[((satellite_count, >=, 7.0), (satellite_count...","[(0.0, 0), (0.0, 0.0), (0.0, 0.0)]"


In [29]:
# Exibir o DataFrame com os erros característicos
df[['Pairs_Then', 'Scope Error by var then']].head()

,Pairs_Then,Scope Error by var then
0,"[((height, >=, 30.0), (height, >=, 30.0)), ((s...","[(0.0, 0.0), (0.0, 0.0)]"
1,"[((height, >=, 30.0), (height, >=, 30.0)), ((s...","[(0.0, 0.0), (0.0, 0.0)]"
2,"[((height, >=, 30.0), (height, >=, 30.0)), ((s...","[(0.0, 0.0), (0.0, 0.0)]"
3,"[((height, >=, 30.0), (height, >=, 30.0)), ((s...","[(0.0, 0.0), (0.0, 0.0)]"
4,"[((height, >=, 30.0), (height, >=, 50.0)), ((s...","[(20.0, 0.0), (0.0, 0.0)]"


In [30]:
# Função para normalizar o erro com base nos limites da variável
def normalize_error(error, variable):
    if variable in limits:
        min_value = limits[variable]['min']
        max_value = limits[variable]['max']
        # Normaliza o erro usando o intervalo da variável
        return error / (max_value - min_value) if max_value - min_value != 0 else 0
    else:
        return error  # Se a variável não estiver no JSON, retorna o erro sem normalização

# Função para normalizar os erros no 'Scope Error by var' usando a variável identificada nos pares
def normalize_scope_error(pairs, errors):
    normalized_errors = []
    for i, pair in enumerate(pairs):
        variable = pair[0][0]  # Identifica a variável a partir do primeiro elemento da tupla do par
        error = errors[i]
        # Normaliza cada erro dentro da lista de erros associados ao par de variáveis
        normalized_error = [normalize_error(e, variable) for e in error]
        normalized_errors.append(normalized_error)
    return normalized_errors

In [32]:
df['Normalized Scope Error by var given'] = df.apply(
    lambda row: normalize_scope_error(row['Pairs_Given'], row['Scope Error by var given']), axis=1
)

df['Normalized Scope Error by var then'] = df.apply(
    lambda row: normalize_scope_error(row['Pairs_Then'], row['Scope Error by var then']), axis=1
)

# Exibir o DataFrame com os erros normalizados
df[['Pairs_Given', 'Scope Error by var given', 'Normalized Scope Error by var given']].head()

,Pairs_Given,Scope Error by var given,Normalized Scope Error by var given
0,"[((satellite_count, >=, 7.0), (satellite_count...","[(0.0, 0), (0.0, 0.0), (0.0, 0.0)]","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0]]"
1,"[((satellite_count, >=, 7.0), (satellite_count...","[(0.0, 0), (10.0, 0.0), (0.0, 0.0)]","[[0.0, 0.0], [0.1, 0.0], [0.0, 0.0]]"
2,"[((satellite_count, >=, 7.0), (satellite_count...","[(1.0, 0), (0.0, 0.0), (0.0, 0.0)]","[[0.06666666666666667, 0.0], [0.0, 0.0], [0.0,..."
3,"[((satellite_count, >=, 7.0), (satellite_count...","[(1.0, 0), (10.0, 0.0), (0.0, 0.0)]","[[0.06666666666666667, 0.0], [0.1, 0.0], [0.0,..."
4,"[((satellite_count, >=, 7.0), (satellite_count...","[(0.0, 0), (0.0, 0.0), (0.0, 0.0)]","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0]]"


In [33]:
# Exibir o DataFrame com os erros normalizados
df[['Pairs_Then', 'Scope Error by var then', 'Normalized Scope Error by var then']].head()

,Pairs_Then,Scope Error by var then,Normalized Scope Error by var then
0,"[((height, >=, 30.0), (height, >=, 30.0)), ((s...","[(0.0, 0.0), (0.0, 0.0)]","[[0.0, 0.0], [0.0, 0.0]]"
1,"[((height, >=, 30.0), (height, >=, 30.0)), ((s...","[(0.0, 0.0), (0.0, 0.0)]","[[0.0, 0.0], [0.0, 0.0]]"
2,"[((height, >=, 30.0), (height, >=, 30.0)), ((s...","[(0.0, 0.0), (0.0, 0.0)]","[[0.0, 0.0], [0.0, 0.0]]"
3,"[((height, >=, 30.0), (height, >=, 30.0)), ((s...","[(0.0, 0.0), (0.0, 0.0)]","[[0.0, 0.0], [0.0, 0.0]]"
4,"[((height, >=, 30.0), (height, >=, 50.0)), ((s...","[(20.0, 0.0), (0.0, 0.0)]","[[0.2, 0.0], [0.0, 0.0]]"


In [34]:
def sum_normalized_errors(normalized_errors):
    # Flatten a lista de listas e soma todos os valores
    return sum(sum(error) for error in normalized_errors)

# Calcular o erro escalar somando todos os valores normalizados
df['Global Normalized Error Sum given'] = df['Normalized Scope Error by var given'].apply(sum_normalized_errors)

df['Global Normalized Error Sum then'] = df['Normalized Scope Error by var then'].apply(sum_normalized_errors)

# Exibir o DataFrame com o valor escalar do erro global normalizado
df[['Pairs_Given', 'Scope Error by var given', 'Normalized Scope Error by var given', 'Global Normalized Error Sum given']]

,Pairs_Given,Scope Error by var given,Normalized Scope Error by var given,Global Normalized Error Sum given
0,"[((satellite_count, >=, 7.0), (satellite_count...","[(0.0, 0), (0.0, 0.0), (0.0, 0.0)]","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0]]",0.000000
1,"[((satellite_count, >=, 7.0), (satellite_count...","[(0.0, 0), (10.0, 0.0), (0.0, 0.0)]","[[0.0, 0.0], [0.1, 0.0], [0.0, 0.0]]",0.100000
2,"[((satellite_count, >=, 7.0), (satellite_count...","[(1.0, 0), (0.0, 0.0), (0.0, 0.0)]","[[0.06666666666666667, 0.0], [0.0, 0.0], [0.0,...",0.066667
3,"[((satellite_count, >=, 7.0), (satellite_count...","[(1.0, 0), (10.0, 0.0), (0.0, 0.0)]","[[0.06666666666666667, 0.0], [0.1, 0.0], [0.0,...",0.166667
4,"[((satellite_count, >=, 7.0), (satellite_count...","[(0.0, 0), (0.0, 0.0), (0.0, 0.0)]","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0]]",0.000000
...,...,...,...,...
5395,"[((satellite_count, >=, 7.0), (satellite_count...","[(7.0, 8.0), (10.0, 90.0), (36.0, 324.0)]","[[0.4666666666666667, 0.5333333333333333], [0....",3.000000
5396,"[((satellite_count, >=, 7.0), (satellite_count...","[(7.0, 8.0), (10.0, 90.0), (180.0, 324.0)]","[[0.4666666666666667, 0.5333333333333333], [0....",3.400000
5397,"[((satellite_count, >=, 7.0), (satellite_count...","[(7.0, 8.0), (10.0, 90.0), (36.0, 324.0)]","[[0.4666666666666667, 0.5333333333333333], [0....",3.000000
5398,"[((satellite_count, >=, 7.0), (satellite_count...","[(7.0, 15.0), (10.0, 90.0), (180.0, 324.0)]","[[0.4666666666666667, 1.0], [0.1, 0.9], [0.5, ...",3.866667


In [36]:
df["Global Normalized Error Sum scenario"] = df["Global Normalized Error Sum given"] + df["Global Normalized Error Sum then"]

In [37]:
df[['Global Normalized Error Sum scenario']].describe()

,Global Normalized Error Sum scenario
count,5400.000000
mean,3.075556
std,0.995933
min,0.000000
25%,2.366667
50%,3.100000
75%,3.766667
max,6.366667


In [38]:
df[['Global Normalized Error Sum scenario','Scenario Similarity']]

,Global Normalized Error Sum scenario,Scenario Similarity
0,0.000000,1.000000
1,0.100000,0.988889
2,0.066667,0.986111
3,0.166667,0.975000
4,0.200000,0.952382
...,...,...
5395,5.000000,0.335000
5396,5.900000,0.333333
5397,5.500000,0.333333
5398,6.366667,0.333333


In [41]:
# Criar o scatter plot usando o Plotly
fig = px.scatter(
    df,
    x='Global Normalized Error Sum scenario',
    y='Scenario Similarity',
    title='Accumulated distance scenario vs Scenario Similarity',
    labels={
        'Global Normalized Error Sum from scenario': 'Accumulated distance',
        'Scenario Similarity': 'Scenario Similarity'
    },
    template='plotly_white'
)

# Exibir o gráfico
fig.show()

Tentar entender pq no maximo foi 4, como eram 3 variáveis formando 6 [chão, tento], imaginei que no pior dos casos, todos os erros de cada um iria tentar a 1, ou seja [1,1], [1,1], [1,1], logo, somando daria 6. Mas no caso foi 4. Tlz nas condicoes nao trazem situacoes desfavorável para todas as variaveis.